In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

## A description of the problem and a discussion of the background. 

#### I used to live in Melbourne when I was studied at Melbourne University. Renting a good apartment is a key problem for every international students like me. For this course, I would like to compare the accommodation in Melbourne with those in Manhattan, US. That would help international students who want to study in New York. I would use the data analysis tools rather than real estate apps and Google. In order to make a comparison and evaluation of the rental options in Manhattan NY, I must set some basis, therefore the apartment in Manhattan must meet the following demands:
### 2 or 3 bedrooms
### Near a metro station in the Manhattan area and within 1.0 mile (1.6 km) radius
### Below 7,000 per month
### Ammenities nearby
### Coffee shops, restaurants, wine stores, gym would be desirable


# Interested Audience

### Students or others who want to live in Manhattan. And this method can also be applied to any other large cities, which provides them a good reference when they are looking for a place to live in.

# 2. Data Section:

### Description of the data and its sources that will be used to solve the problem

### Description of the Data:

#### Deographical information (latitud and longitud)
#### List of Subway metro stations in Manhattan with their address location
#### Apartments with their addresses, price, bedrooms, etc.
#### Other venues nearby 


# How the data will be used to solve the problem

#### Use Foursquare API to list 10 venues on neighborhoods.
#### Use foursquare and geopy data to map the location of subway metro stations , identify the venues and ammenities near each metro station, or explore each subway location separately
#### The location of each apartment and link to the subway locations.
#### Create a map with information like the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station.

#### Data will be from open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.

# Reference of venues around Melbourne University for comparison to Manhattan place

In [2]:
address = 'Swanston Street, Melbourne'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Melbourne home are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Melbourne home are -37.8175134, 144.9675486.


In [3]:
neighborhood_latitude=-37.7936101
neighborhood_longitude=144.9651797

In [4]:
CLIENT_ID='HRQZYFUY242RUNO3LMT3EJRJOVZNUXY5VZYGYHJWON4NYIA2'
CLIENT_SECRET='SIONAK3OSNTUSKJFHRFHKR4X5UMSFS3KYFMZ3KFUMQHIZ0EG'
VERSION=20200218

In [5]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HRQZYFUY242RUNO3LMT3EJRJOVZNUXY5VZYGYHJWON4NYIA2&client_secret=SIONAK3OSNTUSKJFHRFHKR4X5UMSFS3KYFMZ3KFUMQHIZ0EG&v=20200218&ll=-37.7936101,144.9651797&radius=500&limit=100'

In [6]:

results = requests.get(url).json()
#results

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
venues = results['response']['groups'][0]['items']
    
MBnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
MBnearby_venues =MBnearby_venues.loc[:, filtered_columns]

# filter the category for each row
MBnearby_venues['venue.categories'] = MBnearby_venues.apply(get_category_type, axis=1)

# clean columns
MBnearby_venues.columns = [col.split(".")[-1] for col in MBnearby_venues.columns]

MBnearby_venues.head(10)

,name,categories,lat,lng
0,Le Miel et la lune,Café,-37.797014,144.966208
1,Wolf's Lair,Bar,-37.797456,144.967121
2,Princes Park Running Track,Outdoors & Recreation,-37.790727,144.961256
3,Milk the Cow,Cheese Shop,-37.797781,144.967295
4,Heartattack and Vine,Café,-37.797634,144.967290
5,Green Man's Arms,Pub,-37.797077,144.967860
6,Standing Room,Coffee Shop,-37.797294,144.962919
7,Melbourne Uni Footy Oval,Athletics & Sports,-37.793795,144.961562
8,Trotters,Italian Restaurant,-37.797595,144.967539
9,Jimmy Watson's Wine Bar And Restaurant,Wine Bar,-37.797525,144.967388


## Map of Melbourne with venues near residence place - for reference

In [9]:
# create map of Melbourne place  using latitude and longitude values
map_mb = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(MBnearby_venues['lat'], MBnearby_venues['lng'], MBnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_mb)  
    
map_mb